In [ ]:
!pip install -q -U crewai
!pip install -q -U duckduckgo-search
!pip install crewai_tools==0.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.7/308.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.5 MB/s et

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.1 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import tool
import streamlit as st
import re
from dotenv import load_dotenv
# Warning control
import warnings
warnings.filterwarnings('ignore')
my_llm = LLM(
              model='gemini/gemini-1.5-flash',
              api_key=os.getenv("API_KEY")#
            )
from crewai.tools import BaseTool
from langchain_community.tools import DuckDuckGoSearchRun

class MyCustomDuckDuckGoTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "{race} {year} F1 race summary result winner highlights"

    def _run(self, race: str, year:str) -> str:
        duckduckgo_tool = DuckDuckGoSearchRun()

        response = duckduckgo_tool.invoke(f"{race} {year}")

        return response
search_tool = MyCustomDuckDuckGoTool()
searcher = Agent(
    role = 'Formual 1 Content Searcher',
    goal = 'Search for online new articles about {race} for given {year}',
    backstory = '''
                You will be working on creating articles for epaper about {year}.
                You will conduct research online, collect and group information.
                Your work will serve as a basis for the Content Writer.
                ''',
    llm = my_llm,
    verbose=True,
    allow_delegate=False,
    tools=[search_tool]
)
writer = Agent(
    role = 'Formual 1 Content Writer',
    goal = 'Writing a article post about {race} for given {year}',
    backstory = ''''
                Your task is to create a article on {race}, using the data gathered by the Formual 1 Content Searcher as a reference.
                Ensure the article is no longer than 500 words, focusing on the key race results and moments.
                The article should be engaging and informative, maintaining factual accuracy
                ''',
    tools = [search_tool],
    llm=my_llm,
    verbose=True
)
editor = Agent(
    role='Formula 1 Content Editor',
    goal='Ensure the article is factually accurate, clearly structured, and maintains a relaxed yet informative tone, while preserving all key race information.',
    backstory='''
        You are a Formula 1 Content Editor. Your responsibility is to revise articles written by the F1 Content Writer.
        Your main objective is to ensure that the final article:
        - Accurately reflects the outcome of the {race}{year}.
        - Clearly states who won, what happened, and why it mattered.
        - Retains the writer’s original informative and entertaining tone.
        - Avoids speculative or outdated language once the {race}{year} is complete.
        - You must always treat tool outputs—especially race results—as authoritative. Do not contradict or ignore tool-based information, even if it conflicts with prior assumptions.
    ''',
    tools=[search_tool],
    llm=my_llm,
    verbose=True
)
search = Task(
    description='''
        Search and extract information related to the completed Formula 1 {race} {year}.
        - Include final race results, winner, podium finishers, fastest lap, and DNFs.
        - Highlight any major incidents, overtakes, pit strategies, or safety cars.
        - Focus on credible post-race reports or summaries — not pre-race predictions.
    ''',
    agent=searcher,
    expected_output="A structured summary of the Formula 1 {race} {year}, including winner, key events, strategies, and race drama."
)
write = Task(
    description='''
        1. Use the collected content to craft a compelling and engaging article about the Formula 1 {race} {year}.
        2. Naturally integrate SEO-friendly keywords such as "F1 {race}", "{race} {year} winner", and "{race} highlights {year}".
        3. Make the post informative yet conversational, and end with a conclusion or question that encourages reflection or engagement from F1 fans and professionals.
        ''',
    agent=writer,
    expected_output='A well-written 300-word article summarizing the Formula 1 {race} {year} race, suitable for engaging motorsport audiences.'
)
review = Task(
    description=(
        '''
        Review the article about the Formula 1 {race} {year}.
        Your goals:
        - Fix any grammatical, punctuation, or flow-related issues.
        - Ensure the {race}{year} winner and other key outcomes are clearly and accurately presented early in the article.
        - Maintain a relaxed, entertaining, and fan-friendly tone while keeping the content professional.
        - Do not remove or dilute essential facts or conclusions from the writer, especially if the {race}{year} has already happened.
        - Avoid future-tense speculation if the {race}{year} is over.
        - Structure content in clear paragraphs (no bullet points).
        '''
    ),
    agent=editor,
    expected_output='A factually accurate, grammatically correct, well-formatted 300-word article ready for publication. The text must be structured in paragraphs only, with all race outcomes preserved.'
)
crew = Crew(
    agents=[searcher,writer,editor],
    tasks=[search,write,review],
    verbose=1,
    process=Process.sequential
)

def run_agent(race_name, year):
    # Prepare the input dictionary
    template_input = {"race": race_name, "year": year}

    # Run the agent
    result = crew.kickoff(inputs=template_input)

    # Extract title and summary from result.raw
    raw_text = result.raw.strip()
    lines = raw_text.split('\n')
    title = lines[0].replace('## ', '').strip()
    summary = '\n'.join(lines[1:]).strip()

    return title, summary
# Streamlit app setup
st.set_page_config(page_title="F1 Race Summary Agent", layout="centered")
# Reduce top space
st.markdown("""
    <style>
    .block-container {
        padding-top: 1rem;
    }
    </style>
""", unsafe_allow_html=True)
st.title("🏎️ F1 Race Summary Generator")

# User inputs
# race_name = st.text_input("Race Name (e.g., Australian GP):")
# year = st.text_input("Year (e.g., 2025):")
#User inputs (side-by-side layout)
col1, col2 = st.columns([2, 1])  # Adjust width ratio as needed

with col1:
    race_name = st.text_input("Race Name (e.g., Australian GP):")

with col2:
    year = st.text_input("Year (e.g., 2025):")

# Run agent on input
if race_name and year:
    with st.spinner("Agent is generating the summary..."):
        try:
            title, summary = run_agent(race_name, year)
            #st.subheader(title)
            #st.markdown(summary)
            st.markdown("---")
            st.markdown(
                f"""
                <div style="background-color:#1e1e1e; padding: 20px; border-radius: 10px; box-shadow: 0 0 10px rgba(255,255,255,0.1);">
                    <h2 style="color:#ffffff;">🏁 {title}</h2>
                    <p style="color:#f0f0f0; line-height: 1.6; font-size: 16px;">{summary}</pre>
                </div>
                """,
                unsafe_allow_html=True)
        except Exception as e:
            st.error(f"An error occurred while generating the summary:\n{e}")

Overwriting app.py


In [ ]:
!curl https://loca.lt/mytunnelpassword

34.169.201.2

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧your url is: https://pink-seas-deny.loca.lt
^C


In [ ]:
%%writefile app.py
import streamlit as st
import re

def run_agent(user_input):
    return f"Echo from Gemini Agent: {user_input}"

# Streamlit app
st.title("Gemini Pro Agent Demo")

# User input
expression = st.text_input("Which Race You want to generate summary for:", "")

# Evaluate and display result
if expression:
  with st.spinner("Agent is thinking..."):
    result = run_agent(expression)
  st.write(f"Result: {result}")